In [ ]:
!curl -o input.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1089k  100 1089k    0     0  4104k      0 --:--:-- --:--:-- --:--:-- 4141k


In [1]:
with open('input.txt', 'r') as f:
    text = f.read()

print(f"Length of dataset in characters: {len(text)}")

Length of dataset in characters: 1115394


In [2]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [3]:
text_vocab = sorted(list(set(text)))

print("".join(text_vocab))

print(f"Length of vocabulary: {len(text_vocab)}")


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Length of vocabulary: 65


In [5]:
# character to integer mapping. 
stoi = {ch:i for i, ch in enumerate(text_vocab)}
itos = {i:ch for i, ch in enumerate(text_vocab)}

# tiktoken(openai byte-pair-encoding)/sentencepiece(google) are usually used for this in large scale models

# take a string and output a list of integers
encode = lambda s: [stoi[c] for c in s]

# take a list and output a string
decode = lambda l: ''.join([itos[i] for i in l])

encoded_string = encode("Et tu Brutus")
print(encoded_string)
print(decode(encoded_string))

[17, 58, 1, 58, 59, 1, 14, 56, 59, 58, 59, 57]
Et tu Brutus


In [9]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)

In [10]:
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [11]:
# separate the dataset into a train and validation split

n = int(0.9*len(data)) # first 90% will be train, rest val

train_data = data[:n] 
val_data = data[n:]

In [12]:
block_size = 8 # context length: how many characters do we look at when predicting the next one?
train_data[:block_size+1] # example of input(8) and target(9th character) for the first 9 characters in the dataset. We will use this to train the model.

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [13]:
x = train_data[:block_size] # first 8 characters
y = train_data[1:block_size+1] # next 8 characters, which are

for t in range(block_size):
    context = x[:t+1] # the context we have available at time t
    target = y[t] # the next character we want to predict
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [14]:
torch.manual_seed(1337)
batch_size = 4 # num of sequences being processed in parallel
block_size = 8 # maximum context legth for prediction

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split=="train" else val_data
    ix = torch.randint(high=len(data)-block_size, size=(batch_size,))
    x = torch.stack(tensors=[data[i:i+block_size] for i in ix])
    y = torch.stack(tensors=[data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")

print(f"inputs:")
print(xb.shape)
print(xb)

print(f"targets")
print(yb.shape)
print(yb)

print("-" * 50)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1] # the context we have available at time t
        target = yb[b,t] # the next character we want to predict
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
--------------------------------------------------
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56

In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targes are both (B, T) tensor of integers i.e. B = batch, T = Time

        # print(f"idx = {idx}")
        logits = self.token_embedding_table(idx) # (B, T, C) ... C = Channels.   Here, B = 4, T = 8, C = vocab_size = 65
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(input=logits, target=targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            
            logits, loss = self(idx) # get the predictions
            logits = logits[:, -1, :] # focus only on the last time step, this becomes (B, C) 
            probs = F.softmax(logits, dim=-1) # calculate probabilities by applying softmax
            idx_next = torch.multinomial(probs, num_samples=1) # sample from the distribution (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # append sampled index to the running sequence(B, T+1) 
        
        return idx

vocab_size = len(text_vocab)
# print(vocab_size)

m = BigramLanguageModel(vocab_size=vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)



torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [16]:
idx = torch.zeros((1,1), dtype=torch.long)

print(decode(m.generate(idx=idx, max_new_tokens=100)[0].tolist()))


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [17]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(params=m.parameters(), lr=1e-3)

In [18]:
batch_size = 64

for step in range(10000):
    xb, yb = get_batch("train")
    logits, loss = m(xb, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4367170333862305


In [19]:
print(decode(m.generate(idx=idx, max_new_tokens=500)[0].tolist()))


Shomo o st pugino d

ARif w k ithore th, Roue s ped tha okifok, de
Ar.

HESerer ama hatar&Plyour vet hr with is aras Tht; thap
Y:
ENGind hathinthethasiouro oulof I and s l SS:
Iffofo hat t-asoresere ashath fover;

AUMENGHALA:
A:
This;
I t.
NAn thal
Fiotha her owa
Fouidif toury aris ior yoress ane hit in,
O:
LETAUns.
Isat t  st far thasBevellcef f t Bokerdace
My e
TENIris,
G oue, hon buime.
-divEOLALo, warawoofe, M: atre deseeshen tar me ifukeshaceweag t io, d at.
KE: co ctisefang he t veswerde, 


## Self attention - Maths

In [21]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [22]:
xbow = torch.zeros((B,T,C))

for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)
        

In [ ]:
# using matrix multiplication for a weighted aggregation - example

torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(input=a, dim=1, keepdim=True)
b = torch.randint(low=0, high=10, size=(3,2)).float()

c = a @ b

print('a=')
print(a)
print('----')
print('b=')
print(b)
print('----')
print('c=')
print(c)

# in 'c' a simple average of the current and previous data is stored. 

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
----
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
----
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [30]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(dim=1, keepdim=True)

xbow2 = wei @ x # (B, T, T) @ (B, T, C)  --->  (B, T, C)
torch.allclose(xbow, xbow2)

True

In [31]:
# version 3: using softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T, T)
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [ ]:
# version 4: self-attention
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)


# single head performing self attention
head_size = 16
key = nn.Linear(in_features=C, out_features=head_size, bias=False)
query = nn.Linear(in_features=C, out_features=head_size, bias=False)
value = nn.Linear(in_features=C, out_features=head_size, bias=False)

k = key(x)
q = query(x)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)

# creates a lower triangular matrix
# a token can look at itself and the past, but not the future
tril = torch.tril(torch.ones(T, T)) 

# initialize the weight matrix
# wei = torch.zeros((T,T))

# apply the causal mask
wei = wei.masked_fill(tril==0, float('-inf')) # -inf because, softmax(-inf) = 0

# turn scores into probability (each row sums to 1)
wei = F.softmax(wei, dim=-1) 

# apply attention to values
# out = wei @ x

v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [42]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [43]:
out[0]

tensor([[-0.1571,  0.8801,  0.1615, -0.7824, -0.1429,  0.7468,  0.1007, -0.5239,
         -0.8873,  0.1907,  0.1762, -0.5943, -0.4812, -0.4860,  0.2862,  0.5710],
        [ 0.6764, -0.5477, -0.2478,  0.3143, -0.1280, -0.2952, -0.4296, -0.1089,
         -0.0493,  0.7268,  0.7130, -0.1164,  0.3266,  0.3431, -0.0710,  1.2716],
        [ 0.4823, -0.1069, -0.4055,  0.1770,  0.1581, -0.1697,  0.0162,  0.0215,
         -0.2490, -0.3773,  0.2787,  0.1629, -0.2895, -0.0676, -0.1416,  1.2194],
        [ 0.1971,  0.2856, -0.1303, -0.2655,  0.0668,  0.1954,  0.0281, -0.2451,
         -0.4647,  0.0693,  0.1528, -0.2032, -0.2479, -0.1621,  0.1947,  0.7678],
        [ 0.2510,  0.7346,  0.5939,  0.2516,  0.2606,  0.7582,  0.5595,  0.3539,
         -0.5934, -1.0807, -0.3111, -0.2781, -0.9054,  0.1318, -0.1382,  0.6371],
        [ 0.3428,  0.4960,  0.4725,  0.3028,  0.1844,  0.5814,  0.3824,  0.2952,
         -0.4897, -0.7705, -0.1172, -0.2541, -0.6892,  0.1979, -0.1513,  0.7666],
        [ 0.1866, -0.0

In [54]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
# wei = q @ k.transpose(-2,-1)
wei = q @ k.transpose(-2,-1) * head_size**(-0.5) # scaled attention

In [55]:
k.var()

tensor(1.0104)

In [56]:
q.var()

tensor(1.0204)

In [57]:
wei.var()

tensor(1.1053)